## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

In [6]:
from collections import defaultdict
import itertools

def Enumerator(start=0, initial=()):
    return defaultdict(itertools.count(start).__next__, initial)

In [7]:
import glob
import csv
from tqdm.auto import tqdm

id_enumerator = Enumerator()
row_dict = {}
left_id_set = set()
right_id_set = set()
rows_total = 1081 + 1092
clusters_total = 1097

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/Abt-Buy/Abt.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['source'] = 'abt'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/Abt-Buy/Buy.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['source'] = 'buy'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [8]:
true_pair_set = set()

with open(f'{home_dir}/Downloads/Abt-Buy/abt_buy_perfectMapping.csv') as f:
    for row in csv.DictReader(f):
        id_left = id_enumerator[row['idAbt']]
        id_right = id_enumerator[row['idBuy']]
        true_pair_set.add(tuple(sorted([id_left, id_right])))

len(true_pair_set)

1097

In [9]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict

cluster_mapping, cluster_dict = id_pairs_to_cluster_mapping_and_dict(true_pair_set)
len(cluster_mapping)

2173

In [10]:
len(cluster_dict)

1076

In [11]:
cluster_attr = 'cluster_id'
max_cluster_id = max(cluster_mapping.values())

for row_id, row in tqdm(row_dict.items()):
    try:
        row[cluster_attr] = cluster_mapping[row_id]
    except KeyError:
        row[cluster_attr] = max_cluster_id
        max_cluster_id += 1

In [12]:
[row_dict[row_id] for row_id in next(iter(true_pair_set))]

[{'id': 305,
  'name': 'TiVo HD Digital Video Recorder (180 Hour) - TCD652160',
  'description': 'TiVo HD Digital Video Recorder - TCD652160/ Search, Record And Watch Shows In HD/ Record Up To 20 Hours In HD (Or 180 Hours In Standard Definition)/ Record Two Shows At Once In HD/ Replaces Your Cable Box And Works With Over-The-Air Antenna/ USB Connectivity/ Remote Control/ Netflix Instant Streaming/ TiVo Service Required And Sold Separately',
  'price': '',
  'source': 'abt',
  'cluster_id': 305},
 {'id': 1508,
  'name': 'Tivo TIV652160 Tivo High Definition',
  'description': '',
  'manufacturer': 'TiVo',
  'price': '',
  'source': 'buy',
  'cluster_id': 305}]

## Preprocess

In [13]:
attr_list = ['name', 'description', 'price']

In [14]:
import unidecode
from entity_embed.data_utils.one_hot_encoders import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [15]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [16]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [17]:
attr_info_dict = {
    'name': {
        'is_multitoken': True,
        'tokenizer': default_tokenizer,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'description': {
        'is_multitoken': True,
        'tokenizer': default_tokenizer,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,  # compute
        'max_str_len': None,  # compute
    }
}

In [18]:
from entity_embed import build_row_encoder

row_encoder = build_row_encoder(attr_info_dict, row_dict=row_dict)
row_encoder.attr_info_dict

14:56:13 INFO:For attr='name', computing actual alphabet and max_str_len
14:56:13 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
14:56:13 INFO:For attr='name', using actual_max_str_len=16
14:56:13 INFO:For attr='description', computing actual alphabet and max_str_len
14:56:13 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
14:56:13 INFO:For attr='description', using actual_max_str_len=16
14:56:13 INFO:For attr='price', computing actual alphabet and max_str_len
14:56:13 INFO:For attr='price', using actual_max_str_len=14


{'name': OneHotEncodingInfo(is_multitoken=True, tokenizer=<function default_tokenizer at 0x7f3e4426d670>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16),
 'description': OneHotEncodingInfo(is_multitoken=True, tokenizer=<function default_tokenizer at 0x7f3e4426d670>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16),
 'price': OneHotEncodingInfo(is_m

In [19]:
from entity_embed import LinkageDataModule

train_cluster_len = 200
valid_cluster_len = 200
datamodule = LinkageDataModule(
    row_dict=row_dict,
    cluster_attr=cluster_attr,
    row_encoder=row_encoder,
    pos_pair_batch_size=10,
    neg_pair_batch_size=45,
    row_batch_size=16,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=clusters_total - valid_cluster_len - train_cluster_len,
    only_plural_clusters=True,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    log_empty_vals=False,
    random_seed=random_seed
)

## Training

In [20]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    use_mask=True
)

In [21]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 50
early_stop_callback = EarlyStopping(
   monitor='valid_recall_at_0.3',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'abt-buy'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name),
    accumulate_grad_batches=5,
)

14:56:13 INFO:GPU available: True, used: True
14:56:13 INFO:TPU available: None, using: 0 TPU cores
14:56:13 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [22]:
trainer.fit(model, datamodule)

14:56:13 INFO:Train pair count: 214
14:56:13 INFO:Valid pair count: 200
14:56:13 INFO:Test pair count: 704
14:56:18 INFO:
  | Name        | Type           | Params
-----------------------------------------------
0 | blocker_net | BlockerNet     | 1.8 M 
1 | losser      | NTXentLoss     | 0     
2 | miner       | BatchHardMiner | 0     
-----------------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params


1

In [23]:
model.blocker_net.get_signature_weights()

{'name': 0.355547159910202,
 'description': 0.3446945250034332,
 'price': 0.29975828528404236}

## Testing

In [24]:
trainer.test(ckpt_path='best')

14:57:43 INFO:Train pair count: 214
14:57:43 INFO:Valid pair count: 200
14:57:43 INFO:Test pair count: 704



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1_at_0.3': 0.024459921205444667,
 'test_f1_at_0.5': 0.11592106622584976,
 'test_f1_at_0.7': 0.336906584992343,
 'test_f1_at_0.9': 0.14303959131545338,
 'test_pair_entity_ratio_at_0.3': 38.702049780380676,
 'test_pair_entity_ratio_at_0.5': 6.580527086383602,
 'test_pair_entity_ratio_at_0.7': 0.9289897510980967,
 'test_pair_entity_ratio_at_0.9': 0.06808199121522694,
 'test_precision_at_0.3': 0.012389581402387122,
 'test_precision_at_0.5': 0.06240961174769162,
 'test_precision_at_0.7': 0.26004728132387706,
 'test_precision_at_0.9': 0.6021505376344086,
 'test_recall_at_0.3': 0.9492753623188406,
 'test_recall_at_0.5': 0.8130434782608695,
 'test_recall_at_0.7': 0.4782608695652174,
 'test_recall_at_0.9': 0.08115942028985507}
--------------------------------------------------------------------------------


[{'test_precision_at_0.3': 0.012389581402387122,
  'test_recall_at_0.3': 0.9492753623188406,
  'test_f1_at_0.3': 0.024459921205444667,
  'test_pair_entity_ratio_at_0.3': 38.702049780380676,
  'test_precision_at_0.5': 0.06240961174769162,
  'test_recall_at_0.5': 0.8130434782608695,
  'test_f1_at_0.5': 0.11592106622584976,
  'test_pair_entity_ratio_at_0.5': 6.580527086383602,
  'test_precision_at_0.7': 0.26004728132387706,
  'test_recall_at_0.7': 0.4782608695652174,
  'test_f1_at_0.7': 0.336906584992343,
  'test_pair_entity_ratio_at_0.7': 0.9289897510980967,
  'test_precision_at_0.9': 0.6021505376344086,
  'test_recall_at_0.9': 0.08115942028985507,
  'test_f1_at_0.9': 0.14303959131545338,
  'test_pair_entity_ratio_at_0.9': 0.06808199121522694}]

## Testing manually

In [25]:
test_row_dict = datamodule.test_row_dict
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [26]:
embedding_size = model.blocker_net.embedding_size
test_true_pair_set = datamodule.test_true_pair_set

In [27]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [28]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 706 ms, sys: 21.3 ms, total: 727 ms
Wall time: 165 ms


In [29]:
%%time

sim_threshold = 0.3
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 1.12 s, sys: 18.5 ms, total: 1.14 s
Wall time: 287 ms


In [30]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

38.702049780380676

In [31]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.012389581402387122, 0.9492753623188406)

In [32]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

52212

In [33]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

35

In [34]:
cos_similarity = lambda a, b: np.dot(a, b)

In [35]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.26930237,
 {'id': 128,
  'name': 'sony playstation 2 8mb memory card ( 2 pack ) - red / blue finish - 711719706700',
  'description': 'sony playstation 2 8mb memory card ( 2 pack ) - 711719706700 / save and load high scores , positions and replays / magicgate encryption / red / blue finish',
  'price': '$ 34 . 99',
  'source': 'abt',
  'cluster_id': 128},
 {'id': 1109,
  'name': 'sony 7 - 11719 - 70670 - 0 ps2 8 mb memory card 2 - pk - 97067',
  'description': '',
  'manufacturer': 'Sony',
  'price': '',
  'source': 'buy',
  'cluster_id': 128})

(0.101435065,
 {'id': 940,
  'name': 'canon black eos 50d digital slr camera body - eos50dbody',
  'description': "canon black eos 50d digital slr camera body - eos50dbody / 15 . 1 megapixel cmos sensor / digic 4 image processor / 3 . 0 ' clear view lcd",
  'price': '',
  'source': 'abt',
  'cluster_id': 940},
 {'id': 2098,
  'name': "canon eos 50d 15 megapixel slr digital camera with live view & face detection , 3 ' lcd , 6 . 3fps & digic 4 image processor - body only",
  'description': '',
  'manufacturer': 'Canon',
  'price': '$ 1 , 127 . 95',
  'source': 'buy',
  'cluster_id': 940})

(0.22361095,
 {'id': 69,
  'name': 'canon optura / elura accessory kit - accessorykit',
  'description': 'canon optura / elura accessory kit - accessorykit / bp - 2l14 battery / carrying case / mini dv tape .',
  'price': '',
  'source': 'abt',
  'cluster_id': 69},
 {'id': 1331,
  'name': 'canon mini dv camcorder starter kit - 9582a008',
  'description': 'camcorder starter kit',
  'manufacturer': 'Canon',
  'price': '$ 69 . 95',
  'source': 'buy',
  'cluster_id': 69})

(0.059777927,
 {'id': 186,
  'name': 'universal mrf - 350 rf black base station - mrf350',
  'description': 'universal mrf - 350 rf black base station - mrf350 / no more pointing / rf addressable / ir routing / expand operating range up to 100 feet / compatible',
  'price': '$ 250 . 00',
  'source': 'abt',
  'cluster_id': 186},
 {'id': 1437,
  'name': 'universal remote control mrf - 350 addressable narrow band rf base station with rfx - 250 antenna',
  'description': '',
  'manufacturer': 'UNIVERSAL REMOTE CONTROL, INC',
  'price': '$ 174 . 72',
  'source': 'buy',
  'cluster_id': 186})

(0.28009468,
 {'id': 239,
  'name': 'canon fax - jx200 inkjet fax machine - faxjx200',
  'description': 'canon fax - jx200 inkjet fax machine - faxjx200 / automatic document feeder / produce b & w copies with clear , sharp text / one - touch dialing /',
  'price': '$ 78 . 00',
  'source': 'abt',
  'cluster_id': 239},
 {'id': 1446,
  'name': 'canon fax - jx200 fax machine - 1735b002aa',
  'description': 'monochrome copier - 1 . 33 cpm mono - inkjet',
  'manufacturer': 'Canon',
  'price': '',
  'source': 'buy',
  'cluster_id': 239})

(0.22255376,
 {'id': 34,
  'name': 'garmin deluxe carrying case - black finish - 0101023101',
  'description': 'garmin deluxe carrying case - 0101023101 / designed for street pilot iii 2610 / 2650 gps systems / holds your gps system , mounting bracket , cables , batteries and',
  'price': '',
  'source': 'abt',
  'cluster_id': 34},
 {'id': 1117,
  'name': 'garmin canvas deluxe carry case - 010 - 10231 - 01',
  'description': 'top loading - handle - nylon - black',
  'manufacturer': 'Garmin',
  'price': '$ 17 . 45',
  'source': 'buy',
  'cluster_id': 34})

(0.2848063,
 {'id': 904,
  'name': 'olympus stylus 1050 sw black digital camera - stylus1050swb',
  'description': 'olympus stylus 1050 sw black digital camera - stylus1050swb / 10 . 1 megapixel / shockproof / waterproof up to 10 ft ./ freezeproof / 3x optical zoom / 2',
  'price': '',
  'source': 'abt',
  'cluster_id': 904},
 {'id': 2013,
  'name': "olympus stylus 1050 sw 10 megapixel digital camera w / 3x optical zoom , 2 . 7 ' lcd , waterproof , shockproof & freezeproof , tap control , &",
  'description': '',
  'manufacturer': 'Olympus',
  'price': '$ 228 . 95',
  'source': 'buy',
  'cluster_id': 904})

(0.24560267,
 {'id': 590,
  'name': 'samsung 5 disc black home theater system - htz410t',
  'description': 'samsung 5 disc black home theater system - htz410t / 1000 watts total power / usb host play / dolby digital , dolby pro logic ii and dts decoders /',
  'price': '',
  'source': 'abt',
  'cluster_id': 590},
 {'id': 1737,
  'name': 'samsung dvd hm theater sys 5 disc nic - ht - z410 / xaa',
  'description': '',
  'manufacturer': 'Samsung',
  'price': '$ 235 . 14',
  'source': 'buy',
  'cluster_id': 590})

(0.2785387,
 {'id': 791,
  'name': 'polk audio white round two - way in - wall loudspeaker - tc60i',
  'description': "polk audio white round two - way in - wall loudspeaker - tc60i / dynamic balance polymer composite driver / aimable 1 ' silk dome tweeter with neodymium magnet /",
  'price': '$ 299 . 95',
  'source': 'abt',
  'cluster_id': 791},
 {'id': 1373,
  'name': 'polkaudio tci series tc60i in - ceiling loudspeaker - tc60iw',
  'description': '2 - way speaker 125w ( rms ) - white',
  'manufacturer': 'Polk Audio',
  'price': '$ 132 . 83',
  'source': 'buy',
  'cluster_id': 791})

(0.277487,
 {'id': 720,
  'name': 'lg 2 . 0 cu . ft . over - the - range black microwave oven - lmvm2085bk',
  'description': 'lg 2 . 0 cu . ft . over - the - range black microwave oven - lmvm2085bk / warming lamp / glide & spin sliding tray with turntable /',
  'price': '',
  'source': 'abt',
  'cluster_id': 720},
 {'id': 1880,
  'name': 'lg 2 . 0 cu . ft . over the range microwave oven',
  'description': '',
  'manufacturer': 'LG Electronics',
  'price': '',
  'source': 'buy',
  'cluster_id': 720})